In [1]:
import numpy as np
import pandas as pd


X_scaled = np.load("X_scaled_v3.npy")
y_actual_log = np.load("y_actual_log_fixed.npy")
df = pd.read_csv("for_regr_with_descrip.csv")


print(f"Размер X_scaled: {X_scaled.shape}")
print(f"Размер y_actual_log: {y_actual_log.shape}")
print(f"Размер таблицы данных: {df.shape}")



print(f"\nМинимум X_scaled: {np.min(X_scaled)}")
print(f"Максимум X_scaled: {np.max(X_scaled)}")
print(f"Среднее X_scaled: {np.mean(X_scaled)}")
print(f"Дисперсия X_scaled: {np.var(X_scaled)}")

Размер X_scaled: (877, 2027)
Размер y_actual_log: (877,)
Размер таблицы данных: (877, 59)

Минимум X_scaled: -167.81064000001635
Максимум X_scaled: 23968.290999999932
Среднее X_scaled: 4.599451910521244
Дисперсия X_scaled: 12134.427864974188


In [3]:
import pickle
import xgboost as xgb


try:
    model = pickle.load(open("xgb_model.pkl", "rb")) 
except FileNotFoundError:
    model = xgb.XGBRegressor()
    model.load_model("xgb_model_fixed.json")  
print("\nМодель загружена.")


Модель загружена.


In [4]:

expected_features = model.n_features_in_
current_features = X_scaled.shape[1]

if current_features < expected_features:
    missing_features = expected_features - current_features
    print(f"\nДобавляем {missing_features} недостающих признаков (заполняем нулями).")
    X_scaled = np.hstack([X_scaled, np.zeros((X_scaled.shape[0], missing_features))])
    print(f"Новый размер X_scaled: {X_scaled.shape}")

elif current_features > expected_features:
    print("\nОшибка! У нас больше признаков, чем ожидает модель!")

print(f"\nРазмер X_scaled после исправления: {X_scaled.shape}")


Добавляем 1 недостающих признаков (заполняем нулями).
Новый размер X_scaled: (877, 2028)

Размер X_scaled после исправления: (877, 2028)


In [6]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error



X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_actual_log, test_size=0.2, random_state=42)


model = xgb.XGBRegressor(objective="reg:squarederror", n_estimators=100, random_state=42)


print("\nОбучаем XGBoost...")
model.fit(X_train, y_train)


y_pred = model.predict(X_test)


rmse = root_mean_squared_error(y_test, y_pred)
print(f"\nRMSE на тесте: {rmse:.4f}")





Обучаем XGBoost...

RMSE на тесте: 0.1359


In [8]:
model.save_model("xgb_model_v3.json")
print("\nОбученная модель сохранена: xgb_model_v3.json")


Обученная модель сохранена: xgb_model_v3.json


минмакс

In [ ]:
expected_features = model.n_features_in_
current_features = X_scaled.shape[1]

if current_features < expected_features:
    missing_features = expected_features - current_features
    print(f"\nДобавляем {missing_features} недостающих признаков (заполняем нулями).")
    X_scaled = np.hstack([X_scaled, np.zeros((X_scaled.shape[0], missing_features))])
    print(f"Новый размер X_scaled: {X_scaled.shape}")

elif current_features > expected_features:
    print("\nОшибка! У нас больше признаков, чем ожидает модель!")

print(f"\nРазмер X_scaled после исправления: {X_scaled.shape}")

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Загружаем подготовленные признаки (уже очищенные от ненужных колонок)
X_combined = np.load("X_scaled_v3.npy")  # Или используй свою переменную с признаками

# Используем MinMaxScaler вместо StandardScaler
scaler = MinMaxScaler()
X_scaled_alt = scaler.fit_transform(X_combined)

# Сохраняем новый вариант масштабированных данных
np.save("X_scaled_minmax.npy", X_scaled_alt)

print("Файл 'X_scaled_minmax.npy' успешно создан!")

In [ ]:
expected_features = model.n_features_in_
current_features = X_scaled_alt.shape[1]

if current_features < expected_features:
    missing_features = expected_features - current_features
    print(f"\nДобавляем {missing_features} недостающих признаков (заполняем нулями).")
    X_scaled_alt = np.hstack([X_scaled_alt, np.zeros((X_scaled_alt.shape[0], missing_features))])
    print(f"Новый размер X_scaled: {X_scaled_alt.shape}")

elif current_features > expected_features:
    print("\nОшибка! У нас больше признаков, чем ожидает модель!")

print(f"\nРазмер X_scaled_alt после исправления: {X_scaled_alt.shape}")

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Разделяем данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X_scaled_alt, y_actual_log, test_size=0.2, random_state=42)

# Создаём модель XGBoost
model = xgb.XGBRegressor(objective="reg:squarederror", n_estimators=100, random_state=42)

# Обучаем модель
print("\nОбучаем XGBoost...")
model.fit(X_train, y_train)

# Делаем предсказания на тестовых данных
y_pred = model.predict(X_test)

# Оцениваем качество
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"\nRMSE на тесте: {rmse:.4f}")

# Сохраняем модель
model.save_model("xgb_model_fixed.json")
print("\nОбученная модель сохранена: xgb_model_fixed.json")

90% XGBoost + 10% LightGBM-- переобучена
MAE ансамбля: 0.0570
 MAE train: 0.0028
 MAE на test: 0.0570 Минимальное значение y_test: 1.4771212547196624
Максимальное  y_test: 3.2920344359947364


минмакс тоже положительно не влияет

In [ ]:
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

# Загружаем данные
X_scaled = np.load("X_scaled_v3.npy")  # Используем нормализованные данные
y_actual_log = np.load("y_actual_log_fixed.npy")  # Целевая переменная

# Разделяем данные на train/test (80/20)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_actual_log, test_size=0.2, random_state=42)

# Обучаем XGBoost
xgb_model = xgb.XGBRegressor(objective="reg:squarederror", n_estimators=100, random_state=42)
xgb_model.fit(X_train, y_train)

# Обучаем LightGBM
lgb_model = lgb.LGBMRegressor(n_estimators=100, random_state=42)
lgb_model.fit(X_train, y_train)

print("\n✅ XGBoost и LightGBM обучены!")

In [ ]:
# Делаем предсказания на тесте
y_pred_xgb = xgb_model.predict(X_test)
y_pred_lgb = lgb_model.predict(X_test)

# Ансамбль: 90% XGBoost + 10% LightGBM
y_pred_ensemble = 0.9 * y_pred_xgb + 0.1 * y_pred_lgb

# Оценка качества
mae_ensemble = mean_absolute_error(10**y_test, 10**y_pred_ensemble)  # Возвращаем масштаб из логарифма
print(f"\n MAE ансамбля: {mae_ensemble:.4f}")

In [ ]:
import numpy as np

# Проверяем min/max предсказаний
print("\n  Минимальное y_pred_ensemble:", np.min(y_pred_ensemble))
print(" Максимальное y_pred_ensemble:", np.max(y_pred_ensemble))

# Проверяем min/max настоящих значений
print("\n Минимальное y_actual:", np.min(10**y_test))
print(" Максимальное y_actual:", np.max(10**y_test))

In [ ]:
# Делаем предсказания в ЛОГАРИФМЕ
y_pred_log_xgb = xgb_model.predict(X_test)  
y_pred_log_lgb = lgb_model.predict(X_test)  


y_pred_log_ensemble = 0.9 * y_pred_log_xgb + 0.1 * y_pred_log_lgb

mae_log_ensemble = mean_absolute_error(y_test, y_pred_log_ensemble)
print(f"\n MAE ансамбля (log10, как в POSEIDON): {mae_log_ensemble:.4f}")

In [ ]:
# MAE на train
y_pred_train_xgb = xgb_model.predict(X_train)
y_pred_train_lgb = lgb_model.predict(X_train)
y_pred_train_ensemble = 0.9 * y_pred_train_xgb + 0.1 * y_pred_train_lgb

mae_train = mean_absolute_error(y_train, y_pred_train_ensemble)
print(f" MAE (log10) на train: {mae_train:.4f}")

# MAE на test (уже есть, но пересчитаем)
mae_test = mean_absolute_error(y_test, y_pred_ensemble)
print(f" MAE (log10) на test: {mae_test:.4f}")

In [ ]:
print(f"Минимальное значение y_test: {np.min(y_test)}")
print(f"Максимальное значение y_test: {np.max(y_test)}")
print(f"Уникальные значения в y_test: {np.unique(y_test)}")

In [ ]:
import matplotlib.pyplot as plt

errors = y_test - y_pred_ensemble
plt.hist(errors, bins=30, color="blue", alpha=0.7)
plt.xlabel("Ошибка предсказания (log10)")
plt.ylabel("Частота")
plt.title("Распределение ошибок предсказания")
plt.show()

In [ ]:

print(f"Минимум X_train: {np.min(X_train)}")
print(f"Максимум X_train: {np.max(X_train)}")
print(f"Среднее X_train: {np.mean(X_train)}")
print(f"Дисперсия X_train: {np.var(X_train)}")

print(f"Минимум y_train: {np.min(y_train)}")
print(f"Максимум y_train: {np.max(y_train)}")
print(f"Среднее y_train: {np.mean(y_train)}")
print(f"Дисперсия y_train: {np.var(y_train)}")